In [ ]:
import pandas as pd
import all_function as af
#pd.read_excel('database/Biohub authors.xlsx')

import re
import os
import unicodedata
# from datetime import date,time
import datetime
import pandas as pd
import numpy as np

import datacompy
from os.path import exists as file_exists


# Q: how to drop duplicates then combine 'biohub author column?'  df3&df4

In [ ]:

m=list(df.columns)
m.remove('biohub author')


df=df.groupby(m)['biohub author'].apply('; '.join).reset_index()      
#df2.groupby('pmid', as_index=False).agg(sum)
#df=df.drop_duplicates(subset='pmid', keep="last") 

In [ ]:

start=(datetime.date.today() - datetime.timedelta(days=5)).strftime('%Y-%m-%d')
end=(datetime.date.today() -
    datetime.timedelta(days=1)).strftime('%Y-%m-%d')
Keyword='biohub'
df5=af.Bibliometrics_Collect(start)

# 用来重置base dataset的

In [ ]:
# start='2022-09-01'
# end='2022-09-10'
# af.Pubmed_search2(start,end)

start_date='2022-9-1'
end_date='2022-11-10'

import importlib
import all_function as af #import the module here, so that it can be reloaded.
importlib.reload(af)




# start_date='2022-11-1'
# end_date='2022-11-18'

import importlib
import all_function as af #import the module here, so that it can be reloaded.
importlib.reload(af)


start='2022-9-01'
end='2022-11-10'
base=af.Bibliometrics_Collect(start,end)
base.fillna('', inplace=True)
base.to_csv('database/basedb.csv', encoding='utf-8-sig',index=False)

In [ ]:


#for day in range(1,5):
for day in [7,6,5,4,3,2,1]:
    start=(datetime.date.today() - datetime.timedelta(day+4)).strftime('%Y-%m-%d')
    end=(datetime.date.today() - datetime.timedelta(day)).strftime('%Y-%m-%d') 
    df=af.Bibliometrics_Collect(start,end)
    df.fillna('', inplace=True)
    df.to_csv('database/basedb.csv', mode='a', index=False,header=False, encoding='utf-8-sig')
    print('done.',start,':',end)

In [ ]:
base.columns


# 试一下arxiv的api
看了。感觉还是不行。。

# 7. 用bill的来match author

nickname middle lastname  #3 NN-NMI 
 加一下如果有两个middle name且比较长

## 1. 版本1. 只适用于pubmed。 -> 可以改成从指定的index开始循环

In [ ]:

def authormatch_pub(df):
    authors=pd.read_csv('database/pubmed api author.csv', encoding='utf-8-sig')
 
    # get Biohub author names/data

    def ORCID_format (orcid):
        try:
            find_orcid = re.search (r"([0-9]{4})-?([0-9]{4})-?([0-9]{4})-?([0-9]{3}[0-9X])", orcid)
        except:
            find_orcid = False
        if find_orcid:
            formatted_orcid = "https://orcid.org/"+find_orcid.group(1)+"-"+find_orcid.group(2)+"-"+find_orcid.group(3)+"-"+find_orcid.group(4)
        else:
            formatted_orcid = "nan"
        return (formatted_orcid)

    def strip_accents(text):
        try:
            text = unicode(text, 'utf-8')
        except NameError: # unicode is a default on python 3 
            pass

        text = unicodedata.normalize('NFD', text)\
            .encode('ascii', 'ignore')\
            .decode("utf-8")

        return (str(text))

    def text_field_set_null_to_blank (text): 
        if isinstance(text,str):
            return (text)
        else:
            return ("")
        

    condition = set() # for scoring Biohub authors

    Email_address_found = set()

    BiohubAuthors_df = pd.read_excel('database/Biohub authors.xlsx',
                                    dtype = { 
                                        'Middle' : str,
                                        'ORCID' : str,
                                        'Cohort' : str,
                                        'Email-Preferred' : str,
                                        'Email 2' : str},
                                    converters = { 
                                        'Ambiguous initials' : lambda x: np.where(x == True, True, False),
                                        'Ambiguous incomplete full' : lambda x: np.where(x == True, True, False),
                                    })

    BiohubAuthors_df['ORCID'] = BiohubAuthors_df['ORCID'].apply(ORCID_format)

    BiohubAuthors_df['MatchName'] = BiohubAuthors_df['MatchName'].apply(strip_accents)
    BiohubAuthors_df['Last Name'] = BiohubAuthors_df['Last Name'].apply(strip_accents)
    BiohubAuthors_df['First Name'] = BiohubAuthors_df['First Name'].apply(strip_accents)
    BiohubAuthors_df['Nickname'] = BiohubAuthors_df['Nickname'].apply(strip_accents)

    BiohubAuthors_df['Email 2'] = BiohubAuthors_df['Email 2'].apply(text_field_set_null_to_blank)

    BiohubAuthors_df['Length of award'] = BiohubAuthors_df['Length of award'].fillna(0).astype(int)

    BiohubAuthors_df['Award start date'] = BiohubAuthors_df['Award start date'].dt.date
    BiohubAuthors_df['Award end date'] = BiohubAuthors_df['Award end date'].dt.date

    BiohubAuthors_list = BiohubAuthors_df.values.tolist()
    BiohubAuthors_columns = BiohubAuthors_df.columns.tolist()


    List_MatchNames = BiohubAuthors_df['MatchName'].unique().tolist()
    biohub_authors = {} # column names are indexed in BiohubAuthors_columns.index("column name")
    biohub_authors_ORCID = {}
    biohub_authors_email = {}
    biohub_authors_variations = {} 
    biohub_authors_awarddates = {}

    name_match_weight = {
        "FN-NMI" : 3, # "FN-NMI" : first name-no middle initial
        "NN-NMI" : 3, # "NN-NMI" : Nickname-no middle initial
        "FN-MN" : 3,  # "FN-MN" : first name-middle name
        "FI-MN" : 3,  # "FI-MN" : first initial-middle name-(when preferred)
        "FN-MI" : 3,  # "FN-MI" : first name-middle initial
        "FN" : 2,     # "FN" : first name, omitting middle initial
        "NN" : 2,     # "NN" : Nickname-omitting middle initial
        "FI-MI" : 1,  # "FI-MI" : first initial-middle initial
        "FI-NMI" : 1, # "FI-NMI" : first initial-no middle initial
        "FI" : 0      # "FI" : first initial-omitting middle initial
    }

    for i in range (len(BiohubAuthors_df)):
        if not BiohubAuthors_df.loc[i]['MatchName'] in biohub_authors_awarddates:
            biohub_authors_awarddates[BiohubAuthors_df.loc[i]['MatchName']] = []
            if  BiohubAuthors_df.loc[i]['Award start date'] ==  BiohubAuthors_df.loc[i]['Award start date']: # check for null
                biohub_authors_awarddates[BiohubAuthors_df.loc[i]['MatchName']].append(BiohubAuthors_df.loc[i]['Award start date'])
                biohub_authors_awarddates[BiohubAuthors_df.loc[i]['MatchName']].append(BiohubAuthors_df.loc[i]['Award end date'])
            else:
                biohub_authors_awarddates[BiohubAuthors_df.loc[i]['MatchName']].append(datetime.date(2000, 1, 1))
                biohub_authors_awarddates[BiohubAuthors_df.loc[i]['MatchName']].append(datetime.date(3000, 1, 1))


    for row in BiohubAuthors_list:
        MatchName = row[BiohubAuthors_columns.index("MatchName")]
        biohub_authors[MatchName] = row
            
        LastName = row[BiohubAuthors_columns.index("Last Name")].lower()
        FirstName = row[BiohubAuthors_columns.index("First Name")].lower()
        find_bracket = FirstName.find("[") # brackets used to indicate use of first initial as alternate to first name: "J[ames]"
        if find_bracket != -1:
            FirstName = FirstName.replace("[","").replace("]","")
        Nickname = row[BiohubAuthors_columns.index("Nickname")].lower()
        Middle = row[BiohubAuthors_columns.index("Middle")].lower()

        EntryName = LastName+", "+FirstName
        EntryFI = LastName+", "+FirstName[0:1]
        if Middle == "nmi":
            biohub_authors_variations[EntryName] = [MatchName, "FN-NMI"] # first name-no middle initial
            biohub_authors_variations[EntryFI] = [MatchName, "FI-NMI"] # first initial-no middle initial
            if Nickname != FirstName:
                EntryName = LastName+", "+Nickname
                biohub_authors_variations[EntryName] = [MatchName, "NN-NMI"] # Nickname-no middle initial
        else:
            biohub_authors_variations[EntryName] = [MatchName, "FN"] # first name, omitting middle initial
            biohub_authors_variations[EntryFI] = [MatchName, "FI"] # first initial-omitting middle initial
            if Nickname != FirstName:
                EntryName = LastName+", "+Nickname
                biohub_authors_variations[EntryName] = [MatchName, "NN"] # Nickname-omitting middle initial
            if len(Middle) > 1:
                EntryName = LastName+", "+FirstName+" "+Middle
                biohub_authors_variations[EntryName] = [MatchName, "FN-MN"] # first name-middle name
            if find_bracket != -1: 
                EntryName = EntryFI+" "+Middle
                biohub_authors_variations[EntryName] = [MatchName, "FI-MN"] # first initial-middle name-preferred
            EntryName = LastName+", "+FirstName+" "+Middle[0:1]
            biohub_authors_variations[EntryName] = [MatchName, "FN-MI"] # first name-middle initial
            EntryName = EntryFI+" "+Middle[0:1]
            if EntryName not in biohub_authors_variations:
                biohub_authors_variations[EntryName] = [MatchName, "FI-MI"] # first initial-middle initial
        
        email_list = re.findall (r"[a-zA-Z][a-zA-Z0-9_.-]*@[a-zA-Z][a-zA-Z0-9_.-]*", row[BiohubAuthors_columns.index("Email-Preferred")]+" "+row[BiohubAuthors_columns.index("Email 2")])
        for item in email_list:
            item = item.lower()
            biohub_authors_email[item] = row[BiohubAuthors_columns.index("MatchName")]
            
        if row[BiohubAuthors_columns.index("ORCID")] != 'nan':
            biohub_authors_ORCID[row[BiohubAuthors_columns.index("ORCID")]] = row[BiohubAuthors_columns.index("MatchName")]

    biohub_authors_variations_full = {} # includes compressed versions of names, omitting spaces, dashes, apostrophes, etc   

    for key,value in biohub_authors_variations.items():
        biohub_authors_variations_full[key] = value
        compress = key.replace(" ","").replace("-","").replace("\'","").replace(",",", ")
        if key[-2:-1] == " " and key[-3:-2] != ",":
            compress = compress[:-1]+key[-2:] # restore the penultimate space if there is one
        if compress != key:
            biohub_authors_variations_full[compress] = value


    # author_fields notes:
    #    "ForeName" includes middle initials
    #    "Initials" includes first name initial
    #    "Email" field may include email addresses for all authors with listed email addresses, not just the current author
    #    "BiohubAuthor" = "investigator", "intramural" (group/platform leader, etc)
    #    "MatchName" = uniform version of matched name for investigator or group leader, used for joining tables
    #    "MatchType" = Abbreviations for first name, first initial, NMI, etc. in biohub_authors_variations_full
    #    "TrustMatch" = Yes, Maybe, No - based on MatchType (if match based on initials) and if there are matching
    #                   or mis-matching ORCID IDs, email addresses, or campus affiliations
    #    "OrcidMatch" = True if record ORCID matches with Biohub author ORCID; False only if there are values for both that don't match
    #    "EmailMatch" = True if record email matches with Biohub author email
    #                       - ignore if there is a mis-match: Affiliation records for an author occasionally include 
    #                         email addresses for co-authors
    #    "AffiliationMatch" = True if record Affiliation includes a match to Biohub author campus affiliation
    #    "Biohub" -> "Chan Zuckerberg Biohub" or variants found in affiliation or email address
    #    "Biohub-Funding" -> "Chan Zuckerberg Biohub" or variants found in grant list
    authors.drop_duplicates(subset=authors.columns[0:10],inplace=True)
    author_fields = ['AuthorNo', 'pmid', 'name', 'ORCID', 'LastName', 'ForeName', 'Initials',
                    'affiliation', 'ISEmail', 'ISBiohub author','Suffix',
                    'BiohubAuthor',
                    'MatchName',
                    'TrustMatch', 
                    'MatchType',
                    'OrcidMatch',
                    'EmailMatch',
                    'AffiliationMatch',
                    'Biohub', 
                    'Biohub-Funding', 
                    'Stanford',
                    'UCSF',
                    'Berkeley',
                    'Email',  
                    'EqualContrib']

    for col in author_fields:
        if col not in authors.columns.to_list():
            authors[col]=''
    authors = authors[author_fields]
    authors.fillna('', inplace=True)


    # score for Biohub authorship

    for i in range(len(authors)):
        Name = strip_accents(authors.loc[i,"LastName"].lower()+", "+authors.loc[i,"ForeName"].lower())
        MatchName = ""
        MatchType = ""
        if Name in biohub_authors_variations_full:
            MatchName = biohub_authors_variations_full[Name][0]
            MatchType = biohub_authors_variations_full[Name][1] 
        else:
            compress = Name.replace(" ","").replace("-","").replace("\'","").replace(",",", ")
            if Name[-2:-1] == " " and Name[-3:-2] != ",":
                compress = compress[:-1]+Name[-2:] # restore the penultimate space if there is one
            if compress != Name and compress in biohub_authors_variations_full:
                MatchName = biohub_authors_variations_full[compress][0]
                MatchType = biohub_authors_variations_full[compress][1] 
            elif len(authors.loc[i,'ForeName'])>=2:
                if authors.loc[i,'ForeName'][-2] != " " and authors.loc[i,'ForeName'].find(" ") != -1:
                    # if the full middle name is given in the Pubmed record, which may not be present in the Biohub author record
                    forename = authors.loc[i,'ForeName'][:authors.loc[i,'ForeName'].find(" ")+2]
                    Name2 = strip_accents(authors.loc[i,'LastName'].lower()+", "+forename.lower())
                    if Name2 in biohub_authors_variations_full:
                        MatchName = biohub_authors_variations_full[Name2][0]
                        MatchType = biohub_authors_variations_full[Name2][1] 
                        print ("Found full middle name for Biohub author",Name,"PMID",authors.loc[i,'pmid'])
                    else:
                        compress = Name2.replace(" ","").replace("-","").replace("\'","").replace(",",", ")
                        if Name2[-2:-1] == " " and Name[-3:-2] != ",":
                            compress = compress[:-1]+Name[-2:] # restore the penultimate space if there is one
                        if compress != Name2:
                            if compress in biohub_authors_variations_full:
                                MatchName = biohub_authors_variations_full[compress][0]
                                MatchType = biohub_authors_variations_full[compress][1] 
                                print ("Found full middle name for Biohub author",Name,"PMID",authors.loc[i,'pmid'])
                    
                # ToDo - backup if there wasn't a match of the author name: 
                # else: 
                #       if the author record has an ORCID ID, try searching for it in the Biohub Author ORCID ID dictionary
                #       and alert user if there is a match; ditto if there is an email address
                
        if MatchName != "":
            authors.loc[i,'MatchName'] = MatchName
            authors.loc[i,"MatchType"] = MatchType
            authors.loc[i,"BiohubAuthor"] = biohub_authors[MatchName][BiohubAuthors_columns.index("Role")]
            authors.loc[i,"AffiliationMatch"] =  authors.iloc[i,author_fields.index(biohub_authors[MatchName][BiohubAuthors_columns.index("Campus (simple)")])]
            if authors.loc[i,"AffiliationMatch"] != True:
                if len(authors.loc[i,"BiohubAuthor"]) > 0:
                    if authors.loc[i,"Biohub"] == True:
                        authors.loc[i,"AffiliationMatch"] = True 
            orcid_from_record = authors.loc[i,"ORCID"]
            orcid_from_biohub_authors = biohub_authors[MatchName][BiohubAuthors_columns.index("ORCID")]
            if orcid_from_record == orcid_from_biohub_authors:
                authors.loc[i,"OrcidMatch"] = True
            elif len(orcid_from_record) > 0:
                if orcid_from_biohub_authors == "nan":
                    print ("ORCID ID found for \""+MatchName+"\" in Pubmed",authors.loc[i,'pmid'],":\n",orcid_from_record,"\n")
                else:
                    authors.loc[i,"OrcidMatch"] = False
                    print ("Mismatched ORCID ID found for \""+MatchName+"\" in Pubmed",authors.loc[i,'pmid'],"\n  ORCID ID in publication:",orcid_from_record,"\n  ORCID ID from \"Biohub authors.xlsx\" file:",orcid_from_biohub_authors,"\n")
            if str(authors.loc[i,'pmid'])+" "+authors.loc[i,'MatchName'] in Email_address_found:
                authors.loc[i,'EmailMatch'] = True
            
            # set TrustMatch:
            if authors.loc[i,"OrcidMatch"] or authors.loc[i,'EmailMatch'] or authors.loc[i,"Biohub"]:
                authors.loc[i,"TrustMatch"] = "Yes"
                condition.add(1)
            elif authors.loc[i,"AffiliationMatch"] and name_match_weight[MatchType] >= 2:
                authors.loc[i,"TrustMatch"] = "Yes"
                condition.add(2)
            elif authors.loc[i,"AffiliationMatch"] != True and name_match_weight[MatchType] >= 2:
                if biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous incomplete full")] == True and name_match_weight[MatchType] == 2:
                    authors.loc[i,'TrustMatch'] = "Maybe"
                    condition.add(3)
                else:
                    authors.loc[i,'TrustMatch'] = "Yes"
                    condition.add(4)
            elif authors.loc[i,'AffiliationMatch'] and name_match_weight[MatchType] == 1 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == False:
                authors.loc[i,'TrustMatch'] = "Yes"
                condition.add(5)
            elif authors.loc[i,'AffiliationMatch'] and name_match_weight[MatchType] == 0 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == False:
                authors.loc[i,'TrustMatch'] = "Maybe"
                condition.add(6)
                
            elif authors.loc[i,'AffiliationMatch'] and name_match_weight[MatchType] == 1 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == True:
                authors.loc[i,'TrustMatch'] = "Maybe"
                condition.add(7)
            elif authors.loc[i,'AffiliationMatch'] and name_match_weight[MatchType] == 0 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == True:
                authors.loc[i,'TrustMatch'] = "No"
                condition.add(8)
                
            elif authors.loc[i,'AffiliationMatch'] != True and name_match_weight[MatchType] == 1 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == False:
                authors.loc[i,'TrustMatch'] = "Maybe"
                condition.add(9)
            elif authors.loc[i,'AffiliationMatch'] != True and name_match_weight[MatchType] == 0 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == False:
                authors.loc[i,'TrustMatch'] = "No"
                condition.add(10)
            elif authors.loc[i,'AffiliationMatch'] != True and name_match_weight[MatchType] <= 1 and biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")] == True:
                authors.loc[i,'TrustMatch'] = "No"
                condition.add(11)
            else:
                print ("TrustMatch error - didn't match any conditions, Author-PMID",Name,authors[i][author_fields.index("PMID")])
                print (authors.loc[i,'AffiliationMatch'],name_match_weight[MatchType], biohub_authors[MatchName][BiohubAuthors_columns.index("Ambiguous initials")])

            # Suffix match - 
            # for now, no Biohub authors have a suffix; when one does, we'll have to create a field for it; for now:
            
            if len(authors.loc[i,'Suffix'])>0:
                authors.loc[i,'TrustMatch'] = "No"

    authors.to_csv('database/pubmed api author.csv',index=False, encoding='utf-8-sig')		    
    
    
    # save them to dataframe.
    for ind,i in enumerate(authors['pmid']):
        if authors.loc[ind,'TrustMatch']=='Yes':
            if authors.loc[ind,'MatchName'] not in df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'biohub author'].str.lower().replace(', ',' '):
                df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'biohub author'] += authors.loc[ind,'MatchName']
        elif authors.loc[ind,'TrustMatch']=='Maybe':
            df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'possible biohub author'] += authors.loc[ind,'MatchName']
    
    return df

authormatch_pub(df)

## 2.版本2.用于preprint的

除了分号以外的都去掉。比如. , -

if the author record has an ORCID ID, try searching for it in the Biohub Author ORCID ID dictionary
and alert user if there is a match; ditto if there is an email address


经测试。没有last name=nickname的. middle也没有空的

In [ ]:
def authormatch_pre(df):

    def strip_accents(text):
        try:
            text = unicode(text, 'utf-8')
        except NameError: # unicode is a default on python 3 
            pass

        text = unicodedata.normalize('NFD', text)\
            .encode('ascii', 'ignore')\
            .decode("utf-8")

        return (str(text))

    standard=pd.read_excel('database/Biohub authors.xlsx',
                        dtype = { 
                            'Middle' : str,
                            'ORCID' : str,
                            'Cohort' : str,
                            'Email-Preferred' : str,
                            'Email 2' : str},
                        converters = { 
                            'Ambiguous initials' : lambda x: np.where(x == True, True, False),
                            'Ambiguous incomplete full' : lambda x: np.where(x == True, True, False),
                        })
    standard.dropna(how='all', axis=1,inplace=True)	
    for i in standard.columns[2:7]:
        standard[i]=standard[i].apply(strip_accents).str.lower().replace("[","").replace("]","")

    #use a new df to store
    test=standard.iloc[:,2:7]  #后面可以加上2:9, & email相等的话

    from itertools import permutations 

    for ind,i in enumerate(test['Middle']):
        combines_3=[]
        combines_2=[]
        combines_1=[]
        combines_0=[]
        
        if i=='nmi': # no middle name
            # "FN-NMI" : 3  # firstname lastname  & Lastname firstname
            for result in permutations(test.iloc[ind,1:3], 2):
                combines_3.append(" ".join(result))
            
            # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
            for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
                combines_1.append(" ".join(result))
            
            # #   ？      # firstname lastname[0]  & Lastname[0] firstname
            # for result in permutations([test.loc[ind,'First Name'],test.loc[ind,'Last Name'][0]], 2):
            #     combines.append(" ".join(result))
            
            if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
                # NN-NMI
                for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                    combines_3.append(" ".join(result))
                    
                # for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name'][0]], 2):
                #     combines.append(" ".join(result))
                        
        else:
            # FN 
            for result in permutations(test.iloc[ind,1:3], 2):
                combines_2.append(" ".join(result))
                
            # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
            for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
                combines_0.append(" ".join(result))
            
            # FN-MN"            
            combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'Last Name'])
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'First Name'])
            
            # FN-MI
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0])
            
            # FI-MI
            combines_1.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'][0])
            
            if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
                # NN
                for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                    combines_2.append(" ".join(result))
                
            if len(test['Middle'])>1:
                # FN-MN"
                combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'Last Name'])
                combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'First Name'])
                
                # FI-MN
                combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'])
                
        test.loc[ind,'combination3']='; '.join(combines_3)
        test.loc[ind,'combination2']='; '.join(combines_2)
        test.loc[ind,'combination1']='; '.join(combines_1)
        test.loc[ind,'combination0']='; '.join(combines_0)

    test['combination_all']= test['combination0'] + test['combination1'] + test['combination2'] + test['combination3']
    test['combination_23']= test['combination2'] + test['combination3']
    test['combination_01']= test['combination0'] + test['combination1'] 


    df.fillna('', inplace=True)
    preprint_list=['biorxiv','bioRxiv','medrxiv','medRxiv','arxiv','arXiv']
    pre_ind=df[df['journal'].isin(preprint_list)].index
    for ind,i in enumerate(df.loc[pre_ind,'authors']):
        try:
            i=i.split(';')
        except:
            i=i.split(',')

        yes_name_list=list()
        maybe_name_list=list()
        
        for j in i:  # j is every single author name
            j=re.sub(r'[^\w]', ' ', j.strip('#'))
            j=j.replace('  ',' ').strip(' ').replace('-','').lower()

            for ind2 in test.index:
                if j in test.loc[ind2,'combination_01']:
                    x=standard.loc[ind2,'MatchName']
                    maybe_name_list.append(x)
                
                if j in test.loc[ind2,'combination_23']:
                    x=standard.loc[ind2,'MatchName']
                    yes_name_list.append(x)
                    
        df.loc[ind,'possible biohub author']='; '.join(maybe_name_list)
        df.loc[ind,'biohub author']='; '.join(yes_name_list)

    return df


In [ ]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return (str(text))

standard=pd.read_excel('database/Biohub authors.xlsx',
                    dtype = { 
                        'Middle' : str,
                        'ORCID' : str,
                        'Cohort' : str,
                        'Email-Preferred' : str,
                        'Email 2' : str},
                    converters = { 
                        'Ambiguous initials' : lambda x: np.where(x == True, True, False),
                        'Ambiguous incomplete full' : lambda x: np.where(x == True, True, False),
                    })
standard.dropna(how='all', axis=1,inplace=True)	
for i in standard.columns[2:7]:
    standard[i]=standard[i].apply(strip_accents).str.lower().replace("[","").replace("]","")


#储存biohub author所有可能的match
test=standard.iloc[:,2:7]  #后面可以加上2:9, & email相等的话


#经测试。没有last name=nickname的. middle也没有空的

from itertools import permutations 

for ind,i in enumerate(test['Middle']):
    combines_3=[]
    combines_2=[]
    combines_1=[]
    combines_0=[]
    
    if i=='nmi': # no middle name
        # "FN-NMI" : 3  # firstname lastname  & Lastname firstname
        for result in permutations(test.iloc[ind,1:3], 2):
            combines_3.append(" ".join(result))
        
        # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
        for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
            combines_1.append(" ".join(result))
        
        # #   ？      # firstname lastname[0]  & Lastname[0] firstname
        # for result in permutations([test.loc[ind,'First Name'],test.loc[ind,'Last Name'][0]], 2):
        #     combines.append(" ".join(result))
        
        if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
            # NN-NMI
            for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                combines_3.append(" ".join(result))
                
            # for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name'][0]], 2):
            #     combines.append(" ".join(result))
                       
    else:
        # FN 
        for result in permutations(test.iloc[ind,1:3], 2):
            combines_2.append(" ".join(result))
            
        # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
        for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
            combines_0.append(" ".join(result))
        
        # FN-MN"            
        combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'Last Name'])
        combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'First Name'])
        
        # FN-MI
        combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0])
        
        # FI-MI
        combines_1.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'][0])
        
        if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
            # NN
            for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                combines_2.append(" ".join(result))
             
        if len(test['Middle'])>1:
            # FN-MN"
            combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'Last Name'])
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'First Name'])
            
            # FI-MN
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'])
            
    test.loc[ind,'combination3']='; '.join(combines_3)
    test.loc[ind,'combination2']='; '.join(combines_2)
    test.loc[ind,'combination1']='; '.join(combines_1)
    test.loc[ind,'combination0']='; '.join(combines_0)

test['combination_all']= test['combination0'] + test['combination1'] + test['combination2'] + test['combination3']
test['combination_23']= test['combination2'] + test['combination3']
test['combination_01']= test['combination0'] + test['combination1'] 




In [ ]:
df=pd.read_csv('database/basedb.csv', encoding='utf-8-sig')
df.fillna('', inplace=True)

preprint_list=['biorxiv','bioRxiv','medrxiv','medRxiv','arxiv','arXiv']
pre_ind=df[df['journal'].isin(preprint_list)].index
for ind,i in enumerate(df.loc[pre_ind,'authors']):
    try:
        i=i.split(';')
    except:
        i=i.split(',')

    yes_name_list=list()
    maybe_name_list=list()
    
    for j in i:  # j is every single author name
        j=re.sub(r'[^\w]', ' ', j.strip('#'))
        j=j.replace('  ',' ').strip(' ').replace('-','').lower()

        #mat_col=['combination3','combination2','combination1','combination0']
        for ind2 in test.index:
            if j in test.loc[ind2,'combination_01']:
                x=standard.loc[ind2,'MatchName']
                maybe_name_list.append(x)
            
            if j in test.loc[ind2,'combination_23']:
                x=standard.loc[ind2,'MatchName']
                yes_name_list.append(x)
                
    df.loc[ind,'possible biohub author']='; '.join(maybe_name_list)
    df.loc[ind,'biohub author']='; '.join(yes_name_list)


In [ ]:
def Pubmed_search_author(start_date,end_date):
    # start_date=end_date
    author=pd.read_excel('database/Biohub authors.xlsx')
    author_list=author['Last Name']+', '+author['First Name']+ '[FAU]'
    df=pd.DataFrame()

    for term in author_list:
        try:
            dt=af.Pubmed_search2(start_date, end_date,TERM=term,save_AuthorInfo=False)
            dt['biohub author']=term.replace('[FAU]','')
            if isinstance(dt, pd.DataFrame):
                df=pd.concat([df,dt],ignore_index=True)
        except:
            pass

    m=list(df.columns)
    m.remove('biohub author')
    df=df.groupby(m)['biohub author'].apply('; '.join).reset_index()      #df2.groupby('pmid', as_index=False).agg(sum)
    df=df.drop_duplicates(subset='pmid', keep="last") # Q: how to drop duplicates then combine 'biohub author column?'
    return df

end=(datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
df4 = Pubmed_search_author(start_date=end,end_date=end)

In [ ]:
df4

改进一下 
1. middle name search
2. 作者的对应机构

# 10.report
put this in a new page. add side bar.
create a iterative process.  


+ reference

## 1. condition: time period
- check recent report example
just list publication are by intramural author (only from biohub_author.csv. people only from campus simple = biohub)

list publication & summary statics

## 2. preprint compliance report  (around 28min'
number + percentage
only for inverstagtor. organized by inverstagtor MatchName.

statistic is :
    how many publication paper they have , 
    how many publication paper they have they are coressbding author 
    how many publication paper's publication type -> preprint policy apply ( they are research article not a review)
    how many they have preprint & how many they don't
    how many preprint they have \ percentage


# preprint policy
1. 出版类型。
   1. pubmed种有记录 eg  review article 有review这一术语
   2. 有部分的journal只发表review或者别的我们的政策 not apply to的 （check txt file。 这个文件包括全名和缩写。如果一些文章在这些期刊上发表，那么我们的政策不适用
       (~df['journal'].isin(review_list))

2. 当我们运行并生成报告时，我们需要的不仅是
   1. 摘要统计数据 + 一份详细的报告，     【【一个是出版物列表，另一个是摘要统计】】
   2. 可以逐个作者查看第一作者，然后逐个记录，
   3. 对于在首选策略中被称为违规的已发表论文，只需手动管理它们。然后他们可能会说，你知道，当我看记录时，我会说，哦，好吧，出于某种原因，这项政策不适用于此。然后勾选一个字段，表示存在异常。这是个例外。
      1. 对于报告作者栏：只展示前三个作者，其余改成et al  title (year) 但是不重要！后面在搞。先留个函数的框


## 详细报告：
第一行： 加粗的 通讯作者

Author (Year) Title. Journal| Volume|: Pages|. DOI|. PMID: PMID/Accession Number. History: Original Publication|.

List of authors (up to three): LastName, FirstName, Middle Initial ... if there are more than N, than first author only followed by et al. (Year) Title. Journal| Volume|: Pages|. DOI|. PMID: PMID/Accession Number. History: Original Publication|.

Journal = abbreviated version of journal name


# 

preprint compliance report.

In [ ]:
import all_function as af
import pandas as pd

start='2022-7-01'
end='2022-10-30'

# 1. Select author list 
author=pd.read_excel('database/Biohub authors.xlsx')  #  biohub author

# 需要确定下
intramural=author[author['Campus (simple)'] == 'Biohub']
investigator=author.loc[author['Role'] == 'Investigator']

#author.loc[((author['Campus (simple)'] == 'Biohub') & (author['Role'] == 'Investigator')]
df_a=author.loc[(author['Campus (simple)'] == 'Biohub') | (author['Role'] == 'Investigator') ]


# 2. Choose publication & prerpint within specific time period  # 指定时间内的pub & pre
df = pd.read_csv('database/basedb.csv', encoding='utf-8-sig')
df.fillna('', inplace=True)
#df['date'] = df['date'].str.strip('/6/0').str.split(' ')[0]
#df=af.transfer_date_format(df)


# 2.1 Exclude 'review list'
with open('database/list of review journals.txt') as file:
    review_list = [line.rstrip() for line in file]

df=df[~df['journal'].isin(review_list)]

# 2.2 divided into two categories: publication and preprint
#得把date格式改一下然后。换成date
preprint_list=['biorxiv','bioRxiv','medrxiv','medRxiv','arxiv','arXiv']
preprint = df[df['journal'].isin(preprint_list)]
publication = df[~df['journal'].isin(preprint_list)]

df['epost date'] = pd.to_datetime(df['epost date'])  

pre_condition = (df['epost date'] >= start) & (df['epost date'] <= end) & (df['journal'].isin(preprint_list))
op1_pre=df.loc[pre_condition]

pub_condition = (df['epost date'] >= start) & (df['epost date'] <= end) & (~df['journal'].isin(preprint_list))
op1_pub=df.loc[pub_condition]

In [ ]:
op1="""
Biohub intramural research program – Papers published and preprints first-deposited
{start} ~ {end}:
Includes papers, conference proceedings, and preprints published or first-deposited since the last Biohub All-Hands meeting that cite Biohub affiliation or funding and that include a Biohub employee or close, non-investigator affiliate as a co-author (we may easily have missed something, so please feel free to send Bill Burkholder any additions or corrections)
Papers (Research articles, methods papers, reviews, etc.) and conference proceedings:\n
"""

print(sub(op1))

In [ ]:
from docx import Document

d = Document()
d.add_heading('Biohub intramural research program – Papers published and preprints first-deposited\n ', level=1)


paragraph = d.add_paragraph()
paragraph.add_run('dolor').bold
paragraph.add_run(' and some ')

run = paragraph.add_run('dolor22')
run.bold = True

d.save('docx_file.docx')


In [ ]:

run.bold = True
document.add_heading('Biohub intramural research program – Papers published and preprints first-deposited\n ', level=1)

\033[1mBiohub intramural research program – Papers published and preprints first-deposited
\033[1m{start} ~ {end}:
\033[0mIncludes papers, conference proceedings, and preprints published or first-deposited since the last Biohub All-Hands meeting that cite Biohub affiliation or funding and that include a Biohub employee or close, non-investigator affiliate as a co-author (we may easily have missed something, so please feel free to send Bill Burkholder any additions or corrections)
\n\033[1mPapers (Research articles, methods papers, reviews, etc.) and conference proceedings:\n


In [ ]:
# 3.1 Create detail report
# 输出word



## '\033[1m' bold following text 
## '\033[0m' turn to normal
import sys
class safesub(dict):
    def __missing__(self, key):
        return '{' + key + '}'
def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

# 3.1.0 heading
op1="""
\033[1mBiohub intramural research program – Papers published and preprints first-deposited
\033[1m{start} ~ {end}:
\033[0mIncludes papers, conference proceedings, and preprints published or first-deposited since the last Biohub All-Hands meeting that cite Biohub affiliation or funding and that include a Biohub employee or close, non-investigator affiliate as a co-author (we may easily have missed something, so please feel free to send Bill Burkholder any additions or corrections)
\n\033[1mPapers (Research articles, methods papers, reviews, etc.) and conference proceedings:\n
"""

# 3.1.1 publication part
for ind,row in op1_pub.iterrows():
    op1 +='\033[1m'+row['biohub author']+row['possible biohub author']+'\n'+'\033[0m   '+row['title']+str(row['pmid'])+'\n\n'

# 3.1.2 preprint part
op1+='\033[1m'+"Preprints:\n"

for ind,row in op1_pre.iterrows():
    op1+='\033[1m'+row['biohub author']+row['possible biohub author']+'\n'+'\033[0m   '+row['title']+str(row['pmid'])+'\n\n'

print(sub(op1))


## CREATE DF report

In [ ]:

             Biohub staff authors    All authors
Papers (Research articles, methods papers, reviews, etc.) and conference proceedings
Preprints
Total



In [ ]:
b_au=df_a['MatchName'].str.replace(',','').values.tolist()

df[df['biohub author'].isin(b_au)]

In [ ]:
df2=pd.DataFrame(columns =['Biohub staff authors', 'All authors'],index = ['Papers (Research articles, methods papers, reviews, etc.) and conference proceedings', 'Preprints', 'Total'])

df2.iloc[0,1]=op1_pub.shape[0]
df2.iloc[1,1]=op1_pre.shape[0]
df2.iloc[2,1]=df2.iloc[0,1]+df2.iloc[1,1]

df2.iloc[0,0]=op1_pub[(op1_pub['biohub author']!='') | (op1_pub['possible biohub author']!='')].shape[0]
df2.iloc[1,0]=op1_pre[(op1_pre['biohub author']!='') | (op1_pre['possible biohub author']!='')].shape[0]
df2.iloc[2,0]=df2.iloc[0,0]+df2.iloc[1,0]


print("Papers published and preprints first-deposited from",start,'to',end)
df2

### 表3

In [ ]:
df3=df.iloc[df[df['b_au']!=''].index,:]
df3['b_au'] = df3['b_au'].map(lambda x:x.split(', '))  # Q: split by '; '
df3=df3.explode('b_au')
t=pd.DataFrame(df3.groupby('b_au')['record id'].count()).rename(columns={'record id':'Compliance'}) 
t.index.name = None
#t=t[t.index.isin(author['MatchName'])]

p3=pd.DataFrame(columns =['Total articles as corresponding author','Qualifying articles as corresponding author','Qualifying articles as corresponding author with preprints','Compliance'],index = author['MatchName'])
p3.index.name = None
p3['Compliance'].fillna(t['Compliance'])

preprint policy 标准:
    publication type
        1 pubmed: review article
            /PublicationTypeList
            if publication belong to this journal（看bill发的文件）, we can say the preprint policy is not apply
        2 personal curation
        

In [ ]:
from Markdown2docx import Markdown2docx
project = Markdown2docx('Report')
project.eat_soup()
project.save()

author file里的格式是 last_name, first_name

首先。需要是每一行中存在就继续、所以不能用isin 得用循环

其次。match name的大小写、中间是否有逗号得统一
'biohub author','possible biohub author','corresponding author'

现在是全 matchname格式只是小写了而已

In [ ]:
df=af.authormatch_pre(df)
df=af.authormatch_pub(df)
df.to_csv('database/basedb.csv', index=False, encoding='utf-8-sig')

In [ ]:
start='2022-11-1'
end='2022-11-10'
keyword='biohub'
df3 = af.Pubmed_search2(start_date=start, end_date=end,TERM='(zuckerb* AND biohub) OR "cz biohub" OR "czi biohub"',save_AuthorInfo=True)
df4 = af.Pubmed_search_author(start_date=end,end_date=end)

In [ ]:
df4

In [ ]:
df3

In [ ]:
authors=pd.read_csv('database/pubmed api author.csv',encoding='utf-8-sig')		
df = pd.read_csv('database/basedb.csv', encoding='utf-8-sig')      
authors


# save them to dataframe.
df['format biohub author']=''
for ind,i in enumerate(authors['pmid']):
    if authors.loc[ind,'TrustMatch']=='Yes':
        #if authors.loc[ind,'MatchName'] not in df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'biohub author'].str.lower().replace(', ',' '):
        df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'format biohub author'] += '; '+authors.loc[ind,'MatchName']
    if authors.loc[ind,'TrustMatch']=='Maybe':
        df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'possible biohub author'] += '; '+authors.loc[ind,'MatchName']
df['format biohub author']=df['format biohub author'].str.strip('; ')
set(df['format biohub author'])

In [ ]:
authors.loc[ind,'pmid']


In [ ]:
ind=3
authors.loc[ind,'MatchName']

In [ ]:
df

In [ ]:
df6

In [ ]:
import importlib
import all_function as af #import the module here, so that it can be reloaded.
importlib.reload(af)
df6,dfa=af.Pubmed_search2(start_date='2022-11-1',
                 end_date='2022-11-12',TERM='biohub',save_AuthorInfo=False)

# df1=af.BioMedrxiv_Search2(start_date='2022-11-1',
#                 end_date='2022-11-10',
#                 keyword='biohub')

set(af.authormatch_pub(df6)['format biohub author'])


report 增加filter。根据选择可以返回不同的内容
    Campus (simple)	Campus Role	Team	Status
    
    
Tab2:
biohub staff author: Campus (simple)=='Biohub'
    
all author: biohub csv里的

Tab3:
    review article is not Qualifying 

1. Qualifying articles as corresponding author: 	not review paper
2. Qualifying articles as corresponding author with preprints:	
3. Compliance: percentage %.
   
    8 paper = corresponding author & qualifying
        none perprint  Compliance=0
        all have preprint Compliance=100
        4 have preprint Compliance=50


#['Campus (simple)']=='Biohub'
    

In [ ]:
df = pd.read_csv('database/basedb.csv', encoding='utf-8-sig')
df.fillna('', inplace=True)
author=pd.read_csv('database/Biohub authors.csv', encoding='utf-8-sig')  #  biohub author

filter_author=author['MatchName'].str.replace(',','').to_list()


# 2.1 Exclude 'review list'
with open('database/list of review journals.txt') as file:
    review_list = [line.rstrip() for line in file]

df=df[~df['journal'].isin(review_list)]
df['b_au']=df[['biohub author','possible biohub author','corresponding author','format biohub author']].agg('; '.join, axis=1).str.replace('; ; ','; ').str.strip('; ')

ind_list=[]
for ind,i in enumerate(df['b_au']):
    for j in i.split(';'): 
        print(j)
        if j.strip(' ').replace(', ',' ') in filter_author:
            print(j)
            ind_list.append(ind)

print(ind_list)

In [ ]:
df['b_au']

In [ ]:
filter_author

In [ ]:
Charles R Langelier


In [ ]:
# 7. 加两个按钮来输出报告 & report

start=


# 8. 加登录界面。只有特定的人才能修改。
	eg。 只能用biohub的邮箱注册

# 8. author的csv也得保证更新是那种的




把 award date放进author csv






# 加citation - 失败。bill说先搁置

# author match找到missing的之后可以把他replace一下呢

# debug： 只有delete db部分有问题。他变成全delete了

# 11.24 FILTER [Done]

# 11.26 report数据

## 1. 加一个方法。输入名字，输出改好的名字

如何把方法改成：
def (input=名字)
    return format名字，可能等级Yes/No


简单版本

In [ ]:
standard=pd.read_excel('database/Biohub authors.xlsx',
                    dtype = { 
                        'Middle' : str,
                        'ORCID' : str,
                        'Cohort' : str,
                        'Email-Preferred' : str,
                        'Email 2' : str},
                    converters = { 
                        'Ambiguous initials' : lambda x: np.where(x == True, True, False),
                        'Ambiguous incomplete full' : lambda x: np.where(x == True, True, False),
                    })
standard.dropna(how='all', axis=1,inplace=True)	
for i in standard.columns[3:7]:
    standard[i]=standard[i].apply(strip_accents).str.lower().str.replace("[","").str.replace("]","")

#use a new df to store
test=standard.iloc[:,2:7]  

for ind,i in enumerate(test['Middle']):
    combines_3=[]
    combines_2=[]
    combines_1=[]
    combines_0=[]
    
    if i=='nmi': # no middle name
        # "FN-NMI" : 3  # firstname lastname  & Lastname firstname
        for result in permutations(test.iloc[ind,1:3], 2):
            combines_3.append(" ".join(result))
        
        # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
        for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
            combines_1.append(" ".join(result))

        if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
            # NN-NMI
            for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                combines_3.append(" ".join(result))
                
            # for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name'][0]], 2):
            #     combines.append(" ".join(result))
        
    else:
        # FN 
        for result in permutations(test.iloc[ind,1:3], 2):
            combines_2.append(" ".join(result))
            
        # FI-NMI  # firstname[0] lastname  & lastname firstname[0]
        for result in permutations([test.loc[ind,'First Name'][0],test.loc[ind,'Last Name']], 2):
            combines_0.append(" ".join(result))
        
        # FN-MN"            
        combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'Last Name'])
        combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle'][0]+' '+test.loc[ind,'First Name'])
        
        # FN-MI
        combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name']+' '+test.loc[ind,'Middle'][0])
        
        # FI-MI
        combines_1.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'][0])
        
        if (test.loc[ind,'Nickname'] != test.loc[ind,'First Name']):
            # NN
            for result in permutations([test.loc[ind,'Nickname'],test.loc[ind,'Last Name']], 2):
                combines_2.append(" ".join(result))
            
        if len(test['Middle'])>1:
            # FN-MN"
            combines_3.append(test.loc[ind,'First Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'Last Name'])
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'Middle']+' '+test.loc[ind,'First Name'])
            
            # FI-MN
            combines_3.append(test.loc[ind,'Last Name']+' '+test.loc[ind,'First Name'][0]+' '+test.loc[ind,'Middle'])
            
    test.loc[ind,'combination3']='; '.join(combines_3)
    test.loc[ind,'combination2']='; '.join(combines_2)
    test.loc[ind,'combination1']='; '.join(combines_1)
    test.loc[ind,'combination0']='; '.join(combines_0)

test['combination_all']= test['combination0'] + test['combination1'] + test['combination2'] + test['combination3']
test['combination_23']= test['combination2'] + test['combination3']
test['combination_01']= test['combination0'] + test['combination1'] 

test.to_csv('database/biohub author combination.csv',index=False, encoding='utf-8-sig')

In [ ]:
# return old_name, format_name,possible percent
def authormatch_sim(name):
    test=pd.read_csv('database/biohub author combination.csv', encoding='utf-8-sig')
    degree='no'
    format_name=re.sub(r'[^\w]', ' ', name.strip('#'))
    format_name=format_name.replace('  ',' ').strip(' ').replace('-','').lower()
    
    for ind2 in test.index:
        if format_name in test.loc[ind2,'combination_01']:
            format_name=test.loc[ind2,'MatchName']
            degree='maybe'
        
        elif format_name in test.loc[ind2,'combination_23']:
            format_name=test.loc[ind2,'MatchName']
            degree='yes'

    return name,format_name,degree


authormatch_sim('yosef n')

In [ ]:
for ind,i in enumerate(df.loc[pre_ind,'authors']):
    try:
        i=i.split(';')
        
    except:
        i=i.split(',')
    
    for j in i:  
        j=re.sub(r'[^\w]', ' ', j.strip('#'))
        j=j.replace('  ',' ').strip(' ').replace('-','').lower()
        
yosef nir; nir yosef	n yosef; yosef n

In [ ]:
af.authormatch(name)
  for ind in authors['pmid'].index:
        if authors.loc[ind,'TrustMatch']=='Yes':
            #if authors.loc[ind,'MatchName'] not in df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'biohub author'].str.lower().replace(', ',' '):
            df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'format biohub author'] += '; '+authors.loc[ind,'MatchName']
        if authors.loc[ind,'TrustMatch']=='Maybe':
            df.loc[df[df['pmid']==authors.loc[ind,'pmid']].index,'possible biohub author'] += '; '+authors.loc[ind,'MatchName']

    df['format biohub author']=df['format biohub author'].str.strip('; ')
    return df

In [ ]:
import importlib
import all_function as af #import the module here, so that it can be reloaded.
importlib.reload(af)
#import all_function as af

af.authormatch('')

In [ ]:
name=''
test=pd.read_csv('database/biohub author combination.csv', encoding='utf-8-sig')
degree='no'
format_name=re.sub(r'[^\w]', ' ', name.strip('#'))
format_name=format_name.replace('  ',' ').strip(' ').replace('-','').lower()

for ind2 in test.index:
    if format_name in test.loc[ind2,'combination_01']:
        format_name=test.loc[ind2,'MatchName']
        print(format_name,ind2)
        degree='maybe'
    
    elif format_name in test.loc[ind2,'combination_23']:
        format_name=test.loc[ind2,'MatchName']
        print(format_name,1)
        degree='yes'

print(format_name)

In [ ]:
name='Nir Yosef'
biohub_authors_variations_full = np.load('database/biohub_authors_variations_full.npy',allow_pickle=True).item()
name=re.sub(r'[^\w]', ' ', name.strip('#'))
name=name.replace('  ',' ').strip(' ').replace('-','').replace(' ',', ').lower()  # 
Name = strip_accents(name)
print(Name,0)

MatchName = ""
MatchType = ""
if Name in biohub_authors_variations_full:
    print(0)
    MatchName = biohub_authors_variations_full[Name][0]
    MatchType = biohub_authors_variations_full[Name][1] 
else:
    compress = Name.replace(" ","").replace("-","").replace("\'","").replace(",",", ")
    if Name[-2:-1] == " " and Name[-3:-2] != ",":
        compress = compress[:-1]+Name[-2:] # restore the penultimate space if there is one
    if compress != Name and compress in biohub_authors_variations_full:
        MatchName = biohub_authors_variations_full[compress][0]
        MatchType = biohub_authors_variations_full[compress][1] 

print(name,MatchName,MatchType)


In [ ]:
af.authormatch(name='Ada S Y Poon')

In [ ]:
'sarnow, peter'.title()

In [ ]:
np.load('database/biohub_authors_variations_full.npy',allow_pickle=True).item()

In [ ]:
BiohubAuthors_df = pd.read_excel('database/Biohub authors.xlsx',
                                dtype = { 
                                    'Middle' : str,
                                    'ORCID' : str,
                                    'Cohort' : str,
                                    'Email-Preferred' : str,
                                    'Email 2' : str},
                                converters = { 
                                    'Ambiguous initials' : lambda x: np.where(x == True, True, False),
                                    'Ambiguous incomplete full' : lambda x: np.where(x == True, True, False),
                                })

BiohubAuthors_df['MatchName'] = BiohubAuthors_df['MatchName'].apply(strip_accents)
BiohubAuthors_df['Last Name'] = BiohubAuthors_df['Last Name'].apply(strip_accents)
BiohubAuthors_df['First Name'] = BiohubAuthors_df['First Name'].apply(strip_accents)
BiohubAuthors_df['Nickname'] = BiohubAuthors_df['Nickname'].apply(strip_accents)

BiohubAuthors_list = BiohubAuthors_df.values.tolist()
BiohubAuthors_columns = BiohubAuthors_df.columns.tolist()

biohub_authors = {} # column names are indexed in BiohubAuthors_columns.index("column name")
biohub_authors_variations = {} 
biohub_authors_awarddates = {}

name_match_weight = {
    "FN-NMI" : 3, # "FN-NMI" : first name-no middle initial
    "NN-NMI" : 3, # "NN-NMI" : Nickname-no middle initial
    "FN-MN" : 3,  # "FN-MN" : first name-middle name
    "FI-MN" : 3,  # "FI-MN" : first initial-middle name-(when preferred)
    "FN-MI" : 3,  # "FN-MI" : first name-middle initial
    "FN" : 2,     # "FN" : first name, omitting middle initial
    "NN" : 2,     # "NN" : Nickname-omitting middle initial
    "FI-MI" : 1,  # "FI-MI" : first initial-middle initial
    "FI-NMI" : 1, # "FI-NMI" : first initial-no middle initial
    "FI" : 0      # "FI" : first initial-omitting middle initial
}

for row in BiohubAuthors_list:
    MatchName = row[BiohubAuthors_columns.index("MatchName")]
    biohub_authors[MatchName] = row
        
    LastName = row[BiohubAuthors_columns.index("Last Name")].lower()
    FirstName = row[BiohubAuthors_columns.index("First Name")].lower()
    find_bracket = FirstName.find("[") # brackets used to indicate use of first initial as alternate to first name: "J[ames]"
    if find_bracket != -1:
        FirstName = FirstName.replace("[","").replace("]","")
    Nickname = row[BiohubAuthors_columns.index("Nickname")].lower()
    Middle = row[BiohubAuthors_columns.index("Middle")].lower()

    EntryName = LastName+", "+FirstName
    EntryFI = LastName+", "+FirstName[0:1]
    if Middle == "nmi":
        biohub_authors_variations[EntryName] = [MatchName, "FN-NMI"] # first name-no middle initial
        biohub_authors_variations[EntryFI] = [MatchName, "FI-NMI"] # first initial-no middle initial
        if Nickname != FirstName:
            EntryName = LastName+", "+Nickname
            biohub_authors_variations[EntryName] = [MatchName, "NN-NMI"] # Nickname-no middle initial
    else:
        biohub_authors_variations[EntryName] = [MatchName, "FN"] # first name, omitting middle initial
        biohub_authors_variations[EntryFI] = [MatchName, "FI"] # first initial-omitting middle initial
        if Nickname != FirstName:
            EntryName = LastName+", "+Nickname
            biohub_authors_variations[EntryName] = [MatchName, "NN"] # Nickname-omitting middle initial
        if len(Middle) > 1:
            EntryName = LastName+", "+FirstName+" "+Middle
            biohub_authors_variations[EntryName] = [MatchName, "FN-MN"] # first name-middle name
        if find_bracket != -1: 
            EntryName = EntryFI+" "+Middle
            biohub_authors_variations[EntryName] = [MatchName, "FI-MN"] # first initial-middle name-preferred
        EntryName = LastName+", "+FirstName+" "+Middle[0:1]
        biohub_authors_variations[EntryName] = [MatchName, "FN-MI"] # first name-middle initial
        EntryName = EntryFI+" "+Middle[0:1]
        if EntryName not in biohub_authors_variations:
            biohub_authors_variations[EntryName] = [MatchName, "FI-MI"] # first initial-middle initial

biohub_authors_variations_full = {} # includes compressed versions of names, omitting spaces, dashes, apostrophes, etc   

for key,value in biohub_authors_variations.items():
    biohub_authors_variations_full[key] = value
    compress = key.replace(" ","").replace("-","").replace("\'","").replace(",",", ")
    if key[-2:-1] == " " and key[-3:-2] != ",":
        compress = compress[:-1]+key[-2:] # restore the penultimate space if there is one
    if compress != key:
        biohub_authors_variations_full[compress] = value

np.save('database/biohub_authors_variations_full.npy', biohub_authors_variations_full) 



# 11.27 debug

In [ ]:
df = pd.read_csv('database/basedb.csv', encoding='utf-8-sig')
print(df.shape)

start="2022-06-01"
end="2022-11-27"
df['epost date'] = pd.to_datetime(df['epost date'])  
condition = (df['epost date'] >= start) & (df['epost date'] <= end) 
df=df.loc[condition]
df.fillna('', inplace=True)
print(df.shape)

for ind,i in enumerate(df['corresponding author']):
    if i == '':
        break
    i=i.split(';')
    res=[]
    for j in i:  
        j=re.sub(r'[^\w]', ' ', j.strip('#'))
        j=j.replace('  ',' ').strip(' ').replace('-','').lower()
        old_name,new_name,prob=af.authormatch(j)
        res.append(new_name)
    df.loc[ind,'corresponding author'] = '; '.join(m for m in res)
df.fillna('', inplace=True)

print(df.shape)
df.loc[df['journal']=='',:]

In [ ]:
df

In [ ]:
def authormatch(name):
    test=pd.read_csv('database/biohub author combination.csv', encoding='utf-8-sig')
    degree='no'
    format_name=re.sub(r'[^\w]', ' ', name.strip('#'))
    format_name=format_name.replace('  ',' ').strip(' ').replace('-','').lower()
    
    if name=='':
        return name,format_name,degree
    for ind2 in test.index:
        if format_name in test.loc[ind2,'combination_01']:
            format_name=test.loc[ind2,'MatchName']
            degree='maybe'
        
        elif format_name in test.loc[ind2,'combination_23']:
            format_name=test.loc[ind2,'MatchName']
            degree='yes'
    
    return name,format_name,degree


for ind,i in enumerate(df['corresponding author']):
    if i == '':
        continue
    i_list=i.split(';')
    res=[]
    for j in i_list:  
        old_name,new_name,prob=authormatch(j)
        res.append(new_name)
    df.loc[ind,'corresponding author'] = '; '.join(m for m in res)


author=pd.read_csv('database/Biohub authors.csv', encoding='utf-8-sig')  #  biohub author
author['Team']=author['Team'].fillna('NA')
author=author.fillna('')
fauthor_list=author['MatchName'].str.replace(',','').to_list()
df['b_au']=df['format biohub author']
ind_list=[]
biohub_staff_ind=[]
for ind,i in enumerate(df['b_au']):
    i='; '.join(set([j.strip() for j in i.split(';') if j]))
    df.loc[ind,'b_au']=i
    for j in i.split(';'): 
        if j.strip(' ').replace(', ',' ') in fauthor_list:  # only return people who is in filter list
            # Problem: if there are some author not in 'biohub author.xlsx', it will not show their publication
            ind_list.append(ind)   

df=df.iloc[ind_list,:]
pd.option_context('display.max_rows', None, 'display.max_columns', None)

    #print(df)
#df.loc[df['corresponding author']!='',['title','journal','corresponding author']].set_index(['title','journal'])['corresponding author'].str.split("; ", expand=True).stack().reset_index(drop=True, level=-1).reset_index().groupby([0]).size().rename("corresponding author")

# p3=pd.DataFrame(columns =['Total articles as corresponding author','Qualifying articles as corresponding author','Qualifying articles as corresponding author with preprints','Compliance'],index = author['MatchName'])
# p3.index.name = None


# c1=df.loc[df['corresponding author']!='',['title','journal','corresponding author']].set_index(['title','journal'])['corresponding author'].str.split("; ", expand=True).stack().reset_index(drop=True, level=-1).reset_index().groupby([0]).size().rename("corresponding author")
# #c1=c1.groupby(['0']).size().rename(columns={0: "corresponding author"})


# #p3['Total articles as corresponding author']=
# t1=pd.merge(p3,c1,how='left',left_index=True,right_index=True)
# t1

In [313]:
df.fillna('', inplace=True)
p3[p3['Qualifying articles as corresponding author with preprints'].notna()]
#
#df.loc[ (df['corresponding author']!='') & ( df['possible match result']!='' ),:]

,Total articles as corresponding author,Qualifying articles as corresponding author,Qualifying articles as corresponding author with preprints,Compliance
"Banfield, Jill",2.0,2.0,2.0,100.000000
"Fordyce, Polly",16.0,16.0,7.0,228.571429
"Frost, Adam",12.0,12.0,1.0,1200.000000
"Huber, Greg",4.0,4.0,3.0,133.333333
"Marson, Alex",4.0,4.0,2.0,200.000000
"Pollard, Katie",1.0,1.0,1.0,100.000000
"Poon, Ada",4.0,4.0,4.0,100.000000


In [312]:
p3

,Total articles as corresponding author,Qualifying articles as corresponding author,Qualifying articles as corresponding author with preprints,Compliance
"Abate, Adam",NaN,NaN,NaN,NaN
"Altman, Russ",NaN,NaN,NaN,NaN
"Arnaout, Rima",NaN,NaN,NaN,NaN
"Ashley, Euan",NaN,NaN,NaN,NaN
"Banfield, Jill",2.0,2.0,2.0,100.0
...,...,...,...,...
"Yosef, Nir",NaN,NaN,NaN,NaN
"Yu, Bin",NaN,NaN,NaN,NaN
"Zhang, Wenjun",NaN,NaN,NaN,NaN
"Zou, James",NaN,NaN,NaN,NaN


In [ ]:
for ind,i in enumerate(df['corresponding author']):
        if i == '':
            continue 
        i_list=i.split(';')
        res=[]
        for j in i_list:  
            old_name,new_name,prob=af.authormatch(j)
            res.append(new_name)
        df.loc[ind,'corresponding author'] = '; '.join(m for m in res)
    df.fillna('', inplace=True)

In [315]:
authormatch('david j pagliarini')

('david j pagliarini', 'david j pagliarini', 'no')

In [368]:
import importlib
import all_function as af #import the module here, so that it can be reloaded.
importlib.reload(af)

biohub_staff_author=author[author['Campus (simple)'] == 'Biohub']['MatchName'].str.replace(',','').to_list()

# 2. Choose publication & prerpint within specific time period 
df = pd.read_csv('database/basedb.csv', encoding='utf-8-sig')

df['epost date'] = pd.to_datetime(df['epost date'])  
condition = (df['epost date'] >= start) & (df['epost date'] <= end) 
df=df.loc[condition]
df=df.fillna('').reset_index(drop=True)

# format 'corresponding author' column and replace original
for ind,i in enumerate(df['corresponding author']):
    if i == '':
        continue 
    i_list=i.split(';')
    res=[]
    for j in i_list:  
        old_name,new_name,prob=af.authormatch(j)
        res.append(new_name)
    df.loc[ind,'corresponding author'] = '; '.join(m for m in res)
df.fillna('', inplace=True)
#print(df['corresponding author'])

#df['b_au']=df[['biohub author','possible biohub author','format biohub author']].agg('; '.join, axis=1)#.str.replace('; ; ; ','; ').str.replace('; ; ','; ').str.strip('; ')
#df['b_au']=df[['possible biohub author','possible biohub author','format biohub author']].agg('; '.join, axis=1)
df['b_au']=df['format biohub author']

print(df.shape)  
#display(df.tail(5))  

ind_list=[]
biohub_staff_ind=[]
for ind,i in enumerate(df['b_au']):
    i='; '.join(set([j.strip() for j in i.split(';') if j]))
    #print(ind,i)
    df.loc[ind,'b_au']=i
    for j in i.split(';'): 
        if j.strip(' ').replace(', ',' ') in fauthor_list:  # only return people who is in filter list
            # Problem: if there are some author not in 'biohub author.xlsx', it will not show their publication
            ind_list.append(ind)   
        if j.strip(' ').replace(', ',' ') in biohub_staff_author:
            biohub_staff_ind.append(ind)
            
df=df.iloc[list(set(ind_list)),:]

print(df.shape) 
display(df.tail(5))  




(88, 32)
(65, 32)


,record id,save datetime,biohub author,possible biohub author,format biohub author,corresponding author,corresponding author institution,journal,doi,pmid,...,authors2,affiliations list,author - affiliations,published or not,confirm published doi,confirm preprint doi,possible match result,match id,record change number,b_au
77,96.0,2022-11-10,Peter J Turnbaugh,,"Turnbaugh, Peter; Turnbaugh, Peter","Turnbaugh, Peter","Department of Microbiology and Immunology, Uni...",Nature metabolism,10.1038/s42255-022-00684-9,36333489.0,...,,"Department of Microbiology and Immunology, Uni...",Vaibhav Upadhyay: Department of Microbiology a...,,,,,,0,"Turnbaugh, Peter"
78,97.0,2022-11-10,Markita P Landry; Aaron Streets,,"Landry, Markita; Streets, Aaron; Landry, Marki...","Streets, Aaron","Department of Bioengineering, University of Ca...",Scientific reports,10.1038/s41598-022-23054-7,36335145.0,...,,"Biophysics Graduate Group, University of Calif...","Xinyi Zhang: Department of Bioengineering, Uni...",,,,,,0,"Landry, Markita; Streets, Aaron"
79,98.0,2022-11-10,Rob Phillips; Sophie Dumont,,"Dumont, Sophie; Dumont, Sophie",,,eLife,10.7554/eLife.79558,36346735.0,...,,"Biochemistry and Molecular Biophysics Option, ...","Pooja Suresh: Biophysics Graduate Program, Uni...",,,10.1101/2022.04.08.487649,,,0,"Dumont, Sophie"
81,100.0,2022-11-10,Chun Jimmie Ye; Alexander Marson; Kole T Roybal,,"Ye, Jimmie; Marson, Alex; Roybal, Kole; Ye, Ji...",,,Science translational medicine,10.1126/scitranslmed.abm1463,36350984.0,...,,"Department of Microbiology and Immunology, Uni...",Daniel B Goodman: Department of Microbiology a...,,,,,,0,"Ye, Jimmie; Marson, Alex; Roybal, Kole"
83,104.0,2022-11-10,Kerwyn Casey Huang,,"Huang, KC; Huang, KC; Huang, KC",,,Computational and structural biotechnology jou...,10.1016/j.csbj.2022.10.008,36382191.0,...,,"Biophysics Program, Stanford University, Stanf...","Benjamin D Knapp: Biophysics Program, Stanford...",,,,,,0,"Huang, KC"


In [388]:
ind_list=[]
biohub_staff_ind=[]
for ind,i in enumerate(df['b_au']):
    i='; '.join(set([j.strip() for j in i.split(';') if j]))
    df.loc[ind,'b_au']=i
    for j in i.split(';'): 
        if j.strip(' ').replace(', ',' ') in fauthor_list:  # only return people who is in filter list
            # Problem: if there are some author not in 'biohub author.xlsx', it will not show their publication
            ind_list.append(ind)
            break
             
        if j.strip(' ').replace(', ',' ') in biohub_staff_author:
            biohub_staff_ind.append(ind)
            break
print(len([*set(ind_list)]))
len(ind_list)
len(biohub_staff_ind)

84


0

In [377]:
st.write(df['corresponding author'])

感觉possible biohub author','format biohub author 可以删掉。。。
format biohub author  如果按名字搜，会出现一些他们发表论文时不属于biohub的情况，不该纳入考虑

11


In [393]:
pip install aspose-words

ERROR: Could not find a version that satisfies the requirement aspose-words (from versions: none)
ERROR: No matching distribution found for aspose-words
Note: you may need to restart the kernel to use updated packages.


In [394]:

from docx import Document

doc = Document()

title = doc.add_heading('this is title', 1)  
doc.save('test.docx')


In [ ]:
p1 ='##### Papers (Research articles, methods papers, reviews, etc.) and conference proceedings:\n'
for ind,row in p1_pub.iterrows():
    if row['corresponding author'] !='':
        p1 +='- **'+row['b_au']+" of "+row['corresponding author']+"’s lab at "+', '.join(row['corresponding author institution'].split(',')[0:2])+'**\n\n  '+row['title']+' PMID: '+str(row['pmid'])+'\n'
    else:
        p1 +='- **'+row['b_au']+'**\n\n  '+row['title']+' PMID: '+str(row['pmid'])+'\n'

# Record: Prerprint
p1 += "\n\n##### Preprints\n"
for ind,row in p1_pre.iterrows():
    if row['corresponding author'] !='':
        p1 +='- **'+row['b_au']+" of "+row['corresponding author']+"’s lab at "+', '.join(row['corresponding author institution'].split(',')[0:2])+'**\n\n  '+row['title']+' doi: '+str(row['doi'])+'\n'
    else:
        p1 +='- **'+row['b_au']+'**\n\n  '+row['title']+' doi: '+str(row['doi'])+'\n'

p1=p1.replace('; ; ','; ')

st.markdown("### Biohub intramural research program – Papers published and preprints first-deposited \n\nIncludes papers, conference proceedings, and preprints published or first-deposited since the last Biohub All-Hands meeting that cite Biohub affiliation or funding and that include a Biohub employee as a co-author (we may easily have missed something, so please feel free to send Bill Burkholder any additions or corrections)\n")
st.download_button(label='Download Report',data=p1,file_name='Report.md')
st.markdown(p1)    